# 6. データの読み込み、書き出しと ファイル形

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [2]:
%pwd

'/Users/miyamoto/Projects/sample/pydata-book'

## 6.1 テキスト形式のデータの読み書き

In [3]:
!cat ch06/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

- read_csv
 - 型推論

In [4]:
df = pd.read_csv('ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
df["a"]

0    1
1    5
2    9
Name: a, dtype: int64

- 区切り文字の指定

In [6]:
pd.read_table('ch06/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
!cat ch06/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [8]:
pd.read_csv('ch06/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- 明示的にカラム名を指定

In [9]:
pd.read_csv('ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- 明示的に、カラム、インデックス、インデックス名を指定
 - インデックスは特定の列を指定

In [10]:
names = ['a', 'b', 'c', 'd', 'message']
pd_6 = pd.read_csv('ch06/ex2.csv', names=names, index_col='message')

In [11]:
pd_6

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [12]:
print(pd_6.loc["foo"])
print(pd_6.iloc[1])

a     9
b    10
c    11
d    12
Name: foo, dtype: int64
a    5
b    6
c    7
d    8
Name: world, dtype: int64


- 階層型インデックス

In [13]:
!cat ch06/csv_mindex.csv
parsed = pd.read_csv('ch06/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

- 特定の区切り文字がない場合の対応
 - read_tableにホワイトスペースなどのパターンを与える

In [14]:
list(open('ch06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

- sep='\s+' => 正規表現、1文字以上のホワイトスペース

In [15]:
result = pd.read_table('ch06/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


- スキップする行を指定

In [16]:
!cat ch06/ex4.csv
pd.read_csv('ch06/ex4.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- 欠測値の扱い
 - 空文字
 - NA or -1#IND, NULL などがデフォルトとなっている

In [17]:
!cat ch06/ex5.csv
result = pd.read_csv('ch06/ex5.csv')
result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


- na_values = [] で指定

In [19]:
result = pd.read_csv('ch06/ex5.csv', na_values=['NULL',1]) #欠損値として扱う値
result

,something,a,b,c,d,message
0,one,NaN,2,3.0,4,NaN
1,two,5.0,6,NaN,8,world
2,three,9.0,10,11.0,12,foo


- na_values = [] で指定
- ディクショナリでカラムごとに指定

In [20]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ch06/ex5.csv', na_values=sentinels) #dictでも指定できる

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### テキストファイルを少しずつ読み込む

In [21]:
result = pd.read_csv('ch06/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


- 読み込む行数を指定 (多分先頭から数行の感じ)

In [22]:
pd.read_csv('ch06/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


- 少しずつ読む、読み込む行数を指定

In [23]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)
chunker

- tot.order() => 廃止
- tot.sort_values()
- tot.sort_index()

In [24]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0) #グループbyしてカウント取りたい時に、データが多過ぎるから100０行ずつcountした

#tot = tot.order(ascending=False)
tot = tot.sort_values(ascending=False)
#tot.sort_index

In [25]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### テキスト形式でのデータ書き出し

In [26]:
data = pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [27]:
data.to_csv('ch06/out.csv')
!cat ch06/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [28]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


- **Try data.to_json()**

In [29]:
data.to_json()

'{"something":{"0":"one","1":"two","2":"three"},"a":{"0":1,"1":5,"2":9},"b":{"0":2,"1":6,"2":10},"c":{"0":3.0,"1":null,"2":11.0},"d":{"0":4,"1":8,"2":12},"message":{"0":null,"1":"world","2":"foo"}}'

In [30]:
data.to_html


<bound method DataFrame.to_html of   something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo>

- 欠損値の標識を明示的にする

In [31]:
data.to_csv(sys.stdout, na_rep='NULL1') #naを明示的に出力

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL1
1,two,5,6,NULL1,8,world
2,three,9,10,11.0,12,foo


- 明示的にインデックスとヘッダを無効にする

In [32]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


- 出力するカラムを指定

In [33]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


- 日付をインデックスにする

In [34]:
dates = pd.date_range('1/1/2000', periods=7) #便利
ts = Series(np.arange(7), index=dates)
ts.to_csv('ch06/tseries.csv')
!cat ch06/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


- 今一つ、違いがわからない

In [35]:
Series.from_csv('ch06/tseries.csv', parse_dates=True) #日付ぽいところを日付に変えてくれる

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### 区切り文字で区切られた形式を手で操作する

In [36]:
!cat ch06/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [37]:
import csv
f = open('ch06/ex7.csv')

reader = csv.reader(f)

In [38]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [39]:
lines = list(csv.reader(open('ch06/ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

- CSV の方言をサブクラスに実装する

In [40]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [41]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [42]:
#%cat mydata.csv
!cat mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### JSON データ

In [43]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [44]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

In [45]:
asjson = json.dumps(result)

In [46]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,25
1,Katie,33


- Try!
 - data.to_XXXX

### XML と HTML, Web スクレイピング

**NB. The Yahoo! Finance API has changed and this example no longer works**

- Yahoo! Finance APIは変更されており、スクレイピングできない
- オライリーのサイトから電子ブックのデータをスクレイピングしてみる
- <span style="color:red">正直、lxmlでのスクレイピングは使いずらい、beautifulsoup4がイイ！</span>

In [47]:
from lxml.html import parse
#from urllib2 import urlopen
from urllib.request import urlopen

#parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
parsed = parse(urlopen('https://www.oreilly.co.jp/ebook/'))

doc = parsed.getroot()

- findall
 - 引数は、XPath
   - http://qiita.com/rllllho/items/cb1187cec0fb17fc650a
 - 『//』は、descendant-or-selfの省略形です。
   - すなわち起点となるノードの子孫すべての集合を表します。
- a タグをすべて集める

In [48]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x10e37dae8>,
 <Element a at 0x10e37db38>,
 <Element a at 0x10e37db88>,
 <Element a at 0x10e37dbd8>,
 <Element a at 0x10e37dc28>]

- 29番目の a タグの中の参照先(href)を取得

In [49]:
lnk = links[28]
lnk
lnk.get('href')
#lnk.text_content() # Error

'/books/9784873118123'

In [50]:
lnk.items()

[('href', '/books/9784873118123')]

- すべての a タグを集めて、その中から href を取得して、リスト urls に

In [51]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['http://www.adobe.com/jp/downloads/',
 './cart',
 '/catalog/soon.xml',
 '/ebook/new_release.atom',
 './../editors/atom.xml',
 'http://jp.makezine.com/blog/index.xml',
 'http://twitter.com/oreilly_japan',
 '/bookclub/',
 'http://tatsu-zine.com/',
 '/feedback/']

- すべての table タグを集める

In [52]:
tables = doc.findall('.//table')
#calls = tables[1]
#puts = tables[13]
new = tables[0]    # New Lelease
ebook = tables[0]  # eBooks

In [53]:
# rows = calls..findall('.//tr')
rows = ebook.findall('.//tr')

In [54]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [55]:
_unpack(rows[0], kind='th')
_unpack(rows[1], kind='td')

['\r\n                    アイソモーフィックJavaScript\r\n                    \r\n                    \r\n                      Ebook価格2,419円\r\n                      [書誌情報]\r\n                    \r\n                  \r\n                  \r\n                  \r\n                  ',
 '\r\n                    Effective Debugging\r\n                    \r\n                    \r\n                      Ebook価格2,765円\r\n                      [書誌情報]\r\n                    \r\n                  \r\n                  \r\n                  \r\n                  ',
 '\r\n                    サーバーレスシングルページアプリケーション\r\n                    \r\n                    \r\n                      Ebook価格2,246円\r\n                      [書誌情報]\r\n                    \r\n                  \r\n                  \r\n                  \r\n                  ']

In [56]:
from pandas.io.parsers import TextParser

def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

In [57]:
#call_data = parse_options_data(calls)
#put_data = parse_options_data(puts)
#call_data[:10]
ebook_data = parse_options_data(ebook)

In [58]:
ebook_data

,0,1,2
0,\r\n アイソモーフィックJavaScript\r\...,\r\n Effective Debugging\r\...,\r\n サーバーレスシングルページアプリケーション\...


#### lxml.objectify でXMLをパース

In [59]:
#%cd ch06/mta_perf/Performance_XML_Data
%cd ch06/mta_perf

/Users/miyamoto/Projects/sample/pydata-book/ch06/mta_perf


In [60]:
!head -21 Performance_MNR.xml

<?xml  version="1.0" encoding="ISO-8859-1"?>
<PERFORMANCE>
<INDICATOR>
  <INDICATOR_SEQ>28445</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (West of Hudson)</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.
</DESCRIPTION>
  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>1</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>95.00</YTD_TARGET>
  <YTD_ACTUAL>96.90</YTD_ACTUAL>
  <MONTHLY_TARGET>95.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>96.90</MONTHLY_ACTUAL>
</INDICATOR>


In [61]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [62]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [63]:
perf = DataFrame(data)
perf

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


In [64]:
root

<Element PERFORMANCE at 0x10d86d448>

In [65]:
root.get('href')

In [66]:
root.text

## バイナリ形式

In [67]:
#cd ../..

In [68]:
frame = pd.read_csv('ch06/ex1.csv')
frame

FileNotFoundError: File b'ch06/ex1.csv' does not exist

- pickle としてデータを読み書き

In [ ]:
frame.to_pickle('ch06/frame_pickle')

In [ ]:
pd.read_pickle('ch06/frame_pickle')

### HDF5 形式の使用

In [ ]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

In [ ]:
store['obj1']

In [ ]:
store.close()

In [ ]:
os.remove('mydata.h5')

### HTML や Web API を用いた読み書き

In [ ]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)
resp

In [ ]:
data[:5]

In [ ]:
issue_labels = DataFrame(data)
issue_labels

## データベースの読み書き

In [ ]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description

In [ ]:
DataFrame(rows, columns=zip(*cursor.description)[0])